In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Underweight/GSE163903'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Multipathogen analysis of IgA and IgG antigen specificity for selected pathogens in milk produced by women from diverse geographical regions: The INSPIRE Study"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['region: Ethiopia, rural', 'region: Ethiopia, urban', 'region: The Gambia, rural', 'region: The Gambia, urban', 'region: Ghana', 'region: Kenya', 'region: Peru', 'region: Spain', 'region: Sweden', 'region: U.S. California', 'region: U.S. Washington'], 1: ["mother's age: 30", "mother's age: 26", "mother's age: 22", "mother's age: 27", "mother's age: 18", "mother's age: 19", "mother's age: 25", "mother's age: 34", "mother's age: 38", "mother's age: 29", "mother's age: 24", "mother's age: 20", "mother's age: 28", "mother's age: 35", "mother's age: 21", "mother's age: 32", "mother's age: 23", "mother's age: 31", "mother's age:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
# Based on the background information, there's no indication that pure miRNA or methylation data is being used.
# So, let's assume gene expression data is available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 'Underweight' can be inferred from 'bmi code'
trait_row = 7 if len(set(['NORMAL', 'UNDERWEIGHT', 'OVERWEIGHT', 'NA', 'OBESE'])) > 1 else None

# 'age' from 'mother's age'
age_row = 1 if len(set(["30", "26", "22", "27", "18", "19", "25", "34", "38", "29", "24", "20", "28", "35", "21", "32", "23", "31", "36", "40", "17", "42", "33", "NA", "37", "39", "44", "43", "46"])) > 1 else None

# 'gender' from 'infant's sex'
gender_row = 2 if len(set(['0', '1', 'No.Data'])) > 1 else None

# Define conversion functions
def convert_trait(value):
    mapping = {'bmi code: NORMAL': 0, 'bmi code: UNDERWEIGHT': 1, 
               'bmi code: OVERWEIGHT': 0, 'bmi code: OBESE': 0, 'bmi code: NA': None}
    return mapping.get(value.strip(), None)

def convert_age(value):
    try:
        age_str = value.split(":")[1].strip()
        if age_str == 'NA':
            return None
        return float(age_str)
    except:
        return None

def convert_gender(value):
    mapping = {'infant\'s sex: 0': 0, 'infant\'s sex: 1': 1, 'infant\'s sex: No.Data': None}
    return mapping.get(value.strip(), None)

# 3. Save Metadata
save_cohort_info('GSE163903', './preprocessed/Underweight/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Underweight', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Underweight/trait_data/GSE163903.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4989633': [0.0, 30.0, 0.0], 'GSM4989634': [0.0, 26.0, 1.0], 'GSM4989635': [0.0, 22.0, 1.0], 'GSM4989636': [0.0, 27.0, 1.0], 'GSM4989637': [0.0, 18.0, 0.0], 'GSM4989638': [0.0, 19.0, 1.0], 'GSM4989639': [0.0, 25.0, 1.0], 'GSM4989640': [0.0, 18.0, 1.0], 'GSM4989641': [0.0, 18.0, 0.0], 'GSM4989642': [0.0, 25.0, 1.0], 'GSM4989643': [0.0, 30.0, 1.0], 'GSM4989644': [0.0, 34.0, 0.0], 'GSM4989645': [1.0, 38.0, 1.0], 'GSM4989646': [0.0, 29.0, 1.0], 'GSM4989647': [0.0, 24.0, 1.0], 'GSM4989648': [0.0, 20.0, 0.0], 'GSM4989649': [0.0, 20.0, 1.0], 'GSM4989650': [0.0, 25.0, 0.0], 'GSM4989651': [0.0, 26.0, 1.0], 'GSM4989652': [0.0, 20.0, 1.0], 'GSM4989653': [0.0, 22.0, 1.0], 'GSM4989654': [0.0, 18.0, 1.0], 'GSM4989655': [0.0, 22.0, 0.0], 'GSM4989656': [0.0, 28.0, 1.0], 'GSM4989657': [0.0, 35.0, 1.0], 'GSM4989658': [0.0, 21.0, 1.0], 'GSM4989659': [1.0, 28.0, nan], 'GSM4989660': [1.0, 28.0, 0.0], 'GSM4989661': [0.0, 19.0, 1.0], 'GSM4989662': [0.0, 24.0, 0.0], 'GSM4989663': [0.0, 25.0, 1.0], 'GSM498

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['6925_1#52_CLS00168.740', '6925_1#52_CLS01558.996',
       '6925_1#52_CLS02350.482', '6925_1#52_CLS02379.472',
       '6925_1#52_CLS02443.213', '6925_2#88_PhageAmi.725',
       '7622_2#37_CLS00355_a3.492', '7622_4#20_CLS02728.995',
       '7622_4#8_CLS00646.231', '7622_4#8_CLS01888.985',
       '7622_4#8_CLS01926.988', '7622_4#8_CLS02320.214',
       '7622_5#56_CLS01435_a4.477', '7622_5#73_CLS02643.469',
       '7622_5#77_CLS00137.471', '7622_5#77_CLS00206.486',
       '7622_5#77_CLS00320.727', '7622_5#77_CLS00402.994',
       '7622_5#77_CLS00433.227', '7622_5#77_CLS00443.1000'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['CHIKV4263-6k.588', 'CHIKV4263-E3.589', 'CHIKV4263-Capsid.590', 'CHIKV4263-E2.591', 'CHIKV4263-E1.592'], 'Description': ['6k protein', 'E3 protein', 'nonstructural protein PSID', 'nonstructural protein', 'nonstructural protein'], 'Pathogen': ['Chikungunya Virus', 'Chikungunya Virus', 'Chikungunya Virus', 'Chikungunya Virus', 'Chikungunya Virus'], 'Spot_Type': ['IVTT.AG', 'IVTT.AG', 'IVTT.AG', 'IVTT.AG', 'IVTT.AG'], 'Source': ['Antigen Discovery Inc.', 'Antigen Discovery Inc.', 'Antigen Discovery Inc.', 'Antigen Discovery Inc.', 'Antigen Discovery Inc.'], 'SEQUENCE': ['GCCACATACCAAGAGGCTGCGATATACCTGTGGAACGAGCAGCAACCTTTGTTTTGGCTACAAGCCCTTATTCCGCTGGCAGCCCTGATTGTTCTATGCAACTGTCTGAGACTCTTACCATGCTGCTGTAAAACGTTGGCTTTTTTAGCCGTAATGAGCGTCGGTGCCCACACTGTGAGCGCG', 'AGTCTTGCCATCCCAGTTATGTGCCTGTTGGCAAACACCACGTTCCCCTGCTCCCAGCCCCCTTGCACGCCCTGCTGCTACGAAAAGGAACCGGAGGAAACCCTACGCATGCTTGAGGACAACGTCATGAGACCTGGGTACTATCAGCTGCTACAAGCATCCTTAACATGTTCTCCCCACCGCCAGCGACGC', 'ATGGAGTTC

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Determine the identifier key and gene symbol key
    identifier_key = 'ID'
    gene_symbol_key = 'Description'

    # 2. Get the dataframe storing the mapping between probe IDs and genes
    mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping with the 'apply_gene_mapping' function
    gene_data = apply_gene_mapping(gene_data, mapping_df)
